In [2]:
from transformers import BertTokenizerFast, AlbertModel, BertModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
#import hanja
from imblearn.over_sampling import RandomOverSampler
from transformers.optimization import get_cosine_schedule_with_warmup, get_linear_schedule_with_warmup
from transformers import AdamW
from collections import defaultdict
import random
import os

import gc
from tqdm import tqdm

import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from sklearn.metrics import f1_score

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
def preprocessing(title):
    x = title[:]
    x = re.sub('><', ' ', x)
    x = re.sub('[ㄱ-ㅎ!@#$%^&*(),./?1-9]', ' ', x)
    return x.strip()

In [5]:
# Train data
origin_train = pd.read_csv('data/train_ml2.csv')
origin_train = origin_train.dropna()
origin_train = origin_train.loc[:,['sentence', '범죄', '혐오', '선정', '비도덕 아님', '폭력', '비난', '욕설', '차별']]
origin_train.columns = ['sentence', '범죄', '혐오', '선정', '비도덕아님', '폭력', '비난', '욕설', '차별']
#origin_train['sentence'] = origin_train['sentence'].apply(preprocessing)

# Test data
origin_test = pd.read_csv("data/test_ml2.csv")
origin_test = origin_test.loc[:,['sentence', '범죄', '혐오', '선정', '비도덕 아님', '폭력', '비난', '욕설', '차별']]
origin_test.columns = ['sentence', '범죄', '혐오', '선정', '비도덕아님', '폭력', '비난', '욕설', '차별']
#test['sentence'] = test['sentence'].apply(preprocessing)


In [ ]:
# 외부데이터

df = pd.read_table('data/한국어욕설데이터.txt', sep='|', names=['sentence','lbl'])
df = df[df['lbl'] == 0]
df = df.dropna()
df = df[df['sentence'] != ""]

df['범죄'] = 0
df['혐오'] = 0
df['선정'] = 0
df['폭력'] = 0
df['비난'] = 0
df['욕설'] = 0
df['차별'] = 0
df['lbl'] = 1
df.columns = ['sentence', '비도덕아님','범죄','혐오','선정','폭력','비난','욕설','차별']

In [ ]:
origin_train = pd.concat([origin_train, df])# df는 한국어욕설데이터
origin_train['sentence'] = origin_train['sentence'].apply(preprocessing)
origin_train = origin_train[origin_train['sentence'] != ""] # 전처리후 공백이 되면 제거
print("Train Data 길이 :",len(origin_train)) #13,720 -> 이 코드 반복시 Train Data 길이 증가할 수 있음
print("Test Data 길이 :",len(origin_test))



In [4]:
def set_seeds(seed=42) :
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
set_seeds(seed = 42)

NameError: name 'torch' is not defined